In [2]:
import argparse
from ROOT import TFile, TH2D
import numpy as np
from astropy.io import fits

Welcome to JupyROOT 6.24/06


In [4]:
Fmap = TFile.Open("/data/home/cwy/Science/3MLWCDA/data/dust-allsky-2.root")
Hmap = Fmap.Get("h2")
NbinsX = int(Hmap.GetNbinsX())
NbinsY = int(Hmap.GetNbinsY())
WbinX = Hmap.GetXaxis().GetBinWidth(1)
WbinY = Hmap.GetYaxis().GetBinWidth(1)
X0 = Hmap.GetXaxis().GetBinLowEdge(1)
Y0 = Hmap.GetYaxis().GetBinLowEdge(1)
print("NbinsX=%d, NbinsY=%d"%(NbinsX, NbinsY))
print("WbinX=%.2lf, WbinY=%.2lf"%(WbinX, WbinY))
print("X0=%.2lf, Y0=%.2lf"%(X0, Y0))

Data = np.zeros((NbinsY, NbinsX))
for ix in range(0, NbinsX):
    for iy in range(0, NbinsY):
        Data[iy][ix] = Hmap.GetBinContent(ix+1, iy+1)

# 创建一个PrimaryHDU对象，并将数据存储在其中
hdu = fits.PrimaryHDU(Data)
hdr = hdu.header
hdr.set('CRVAL1', X0-WbinX/2)
hdr.set('CDELT1', WbinX)
hdr.set('CRPIX1', 1)
hdr.set('CTYPE1', "RA---TAN")
hdr.set('CRVAL2', Y0-WbinY/2)
hdr.set('CDELT2', WbinY)
hdr.set('CRPIX2', 1)
hdr.set('CTYPE2', "DEC--TAN")
hdr.set('Content', 'Source significance map of 25TeV<E<100TeV')
# 写入头信息
print(hdu.header)
# 将HDU对象保存为FITS文件
hdu.writeto('dust_fullsky.fits', overwrite=True)

NbinsX=3600, NbinsY=1800
WbinX=0.10, WbinY=0.10
X0=0.00, Y0=-90.00
SIMPLE  =                    T / conforms to FITS standard                      BITPIX  =                  -64 / array data type                                NAXIS   =                    2 / number of array dimensions                     NAXIS1  =                 3600                                                  NAXIS2  =                 1800                                                  EXTEND  =                    T                                                  CRVAL1  =                -0.05                                                  CDELT1  =                  0.1                                                  CRPIX1  =                    1                                                  CTYPE1  = 'RA---TAN'                                                            CRVAL2  =               -90.05                                                  CDELT2  =                  0.1                       

In [6]:
import numpy as np
from astropy.io import fits
from astropy.wcs import WCS
import ROOT

def create_fits_from_root(root_file_path, hist_name, output_fits_name, content_desc):
    """
    从ROOT文件生成FITS文件。

    Parameters:
        root_file_path (str): ROOT文件路径
        hist_name (str): ROOT文件中TH2D直方图的名称
        output_fits_name (str): 输出FITS文件的名称
        content_desc (str): FITS头文件中的内容描述

    Returns:
        None
    """
    # 打开ROOT文件并获取TH2D直方图
    Fmap = ROOT.TFile.Open(root_file_path)
    Hmap = Fmap.Get(hist_name)

    # 获取直方图的基本信息
    NbinsX = int(Hmap.GetNbinsX())
    NbinsY = int(Hmap.GetNbinsY())
    WbinX = Hmap.GetXaxis().GetBinWidth(1)
    WbinY = Hmap.GetYaxis().GetBinWidth(1)
    X0 = Hmap.GetXaxis().GetBinLowEdge(1)
    Y0 = Hmap.GetYaxis().GetBinLowEdge(1)

    print("NbinsX=%d, NbinsY=%d" % (NbinsX, NbinsY))
    print("WbinX=%.2lf, WbinY=%.2lf" % (WbinX, WbinY))
    print("X0=%.2lf, Y0=%.2lf" % (X0, Y0))

    # 将ROOT直方图的数据导出为numpy数组
    Data = np.zeros((NbinsY, NbinsX))
    for ix in range(NbinsX):
        for iy in range(NbinsY):
            Data[iy][ix] = Hmap.GetBinContent(ix + 1, iy + 1)

    # 创建FITS头文件并设置关键字
    wcs = WCS(naxis=2)
    wcs.wcs.crpix = [NbinsX / 2 + 0.5, NbinsY / 2 + 0.5]  # 修改为中心像素
    wcs.wcs.cdelt = [WbinX, WbinY]
    wcs.wcs.crval = [X0 + (NbinsX * WbinX) / 2, Y0 + (NbinsY * WbinY) / 2]  # 修改参考坐标为中心
    wcs.wcs.ctype = ["RA---CAR", "DEC--CAR"]  # 尝试使用 CAR 投影

    # 创建FITS文件并写入数据和头文件
    hdu = fits.PrimaryHDU(data=Data, header=wcs.to_header())
    hdr = hdu.header
    hdr.set('Content', content_desc)
    print(hdr)
    hdu.writeto(output_fits_name, overwrite=True)

# 使用函数
root_file_path = "/data/home/cwy/Science/3MLWCDA/data/dust-allsky-2.root"
hist_name = "h2"
output_fits_name = "dust_fullsky.fits"
content_desc = "Source significance map of 25TeV<E<100TeV"

create_fits_from_root(root_file_path, hist_name, output_fits_name, content_desc)


NbinsX=3600, NbinsY=1800
WbinX=0.10, WbinY=0.10
X0=0.00, Y0=-90.00
SIMPLE  =                    T / conforms to FITS standard                      BITPIX  =                  -64 / array data type                                NAXIS   =                    2 / number of array dimensions                     NAXIS1  =                 3600                                                  NAXIS2  =                 1800                                                  WCSAXES =                    2 / Number of coordinate axes                      CRPIX1  =               1800.5 / Pixel coordinate of reference point            CRPIX2  =                900.5 / Pixel coordinate of reference point            CDELT1  =                  0.1 / [deg] Coordinate increment at reference point  CDELT2  =                  0.1 / [deg] Coordinate increment at reference point  CUNIT1  = 'deg'                / Units of coordinate increment and value        CUNIT2  = 'deg'                / Units of coordinate 